
W00005371 河南省洛众商贸有限公司 0030011244
W00013686 南通荣尚酒业有限公司 0030018687
W00014676 广东瑞丰和酒业有限公司潮州仓 0030019378
W00008121 深圳市佰兴泰贸易有限公司 0030012653

分析时间范围：
2023.1.1~2023.12.31

先进先出的原则，库存可以用满7个月，如1月进的货8月失效

2024.11.29 产品库存只看四个brand family 'BUD','HBI','HBO','HKOW';或者isp_sku不为空
2024.11.29 时间范围2021.1~2024.10
2024.12.2  每个brand family一个保质期，保险先把保质期设置成9个月的 

In [1]:
import pandas as pd
import re
from openpyxl import load_workbook
from datetime import datetime
import yaml
import warnings
warnings.filterwarnings('ignore')
from fetch_data import *
from get_hl_from_skuname import *

In [13]:
def make_monthly_horizontal_df(lst, name):
    df = pd.DataFrame(lst, columns=[name]).T
    df.columns = [x + 1 for x in df.columns]
    df = df.add_prefix('month_')
    return df


def get_initial_inv(data, **params):
    ws_payer_code = params['ws_payer_code']
    ws_id_name = params['ws_id_name']
    ws_inv = data.query(f"{ws_id_name} == '{ws_payer_code}'")
    if params['agg_col'] == 'hl':
        ws_inv[params['sku_name']] = ws_inv[params['sku_name']].str.upper()
        ws_inv[params['sku_name']] = ws_inv[params['sku_name']].apply(lambda x: re.sub(r'\s|[\d]%', '', x))
        sku_name_list = list(ws_inv[params['sku_name']].unique())
        sku_hl_mapping_table = get_sku_hl_mapping_table(sku_name_list)
        ws_inv = pd.merge(ws_inv, sku_hl_mapping_table[['sku_name', 'hl']], left_on=params['sku_name'], right_on='sku_name')
    else:
        sku_hl_mapping_table = pd.DataFrame()                                                                          
    ws_inv_agg = ws_inv.groupby([ws_id_name])[params['agg_col']].sum()
    return ws_inv_agg, sku_hl_mapping_table


def get_num_list(stw, **params):
    # STW/STR 12个月的值
    ws_stw = stw.query(f"{params['ws_id_name']} == {params['ws_payer_code']}")
    if params['agg_col'] == 'hl':   # ws stw表里的agg_col是stw_hl
        ws_stw[params['sku_name']] = ws_stw[params['sku_name']].str.upper()
        ws_stw[params['sku_name']] = ws_stw[params['sku_name']].apply(lambda x: re.sub(r'\s|[\d]%', '', x))
        sku_name_list = list(ws_stw[params['sku_name']].unique())
        sku_hl_mapping_table = get_sku_hl_mapping_table(sku_name_list)
        ws_stw = pd.merge(ws_stw, sku_hl_mapping_table[['sku_name', 'hl']], left_on=params['sku_name'], right_on='sku_name')
    else:
        sku_hl_mapping_table = pd.DataFrame()                                                                        
    ws_stw_agg = ws_stw.groupby([params['month_col']])[params['agg_col']].sum()       
    ws_stw_agg = ws_stw_agg.reindex(stw_yr_month_index).fillna(0)
    ws_stw_lst = ws_stw_agg.to_list()
    return ws_stw_lst, sku_hl_mapping_table

In [3]:
def get_stw_monthly_balance(begin_inv, stw: list, out_str: list, inv_expire_month=6, simu_months=12):
    """
    begin_inv:202212期末库存=2023年1月期初库存
    stw: 2023年12个月的stw进货量
    out_str:2023年12个月的出货量
    simu_months: 模拟的月份数，默认是1年即12个月
    result: 期初库存每个月的结余
    """
#     # 测试数据
#     begin_inv = 80
#     stw = [80]*2 + [100]*10     # 进货
#     out_str = [80]*7 + [80]*5

    begin_inv_remain = begin_inv
    stw_remain = stw.copy()             #  截止到当前月每个月的进货量还剩多少                
    stw_remain_monthly = []             # 每个月的结果合集
    begin_inv_remain_monthly = []       # 每个月的结果合集
    #     begin_inv_remain_lst = [None] * 12
    #     stw_remain_list = []        # 每个月进货stw的剩余
    str_remain_list = []
    for i in range(simu_months):
        stw_remain_list = []
        if (out_str[i] <= begin_inv_remain) and (i <= (inv_expire_month - 1)):     # 期初库存能扣到6月
            begin_inv_remain -= out_str[i]  # 期初（年初）库存的剩余
            stw_remain_list = stw_remain.copy()     # 每个月stw的剩余
            stw_remain_list = [stw_remain_list[stw_month] if stw_month <= i else None for stw_month in range(simu_months) ]
            str_remain = 0
            str_remain_list.append(str_remain)
        else:
            if (begin_inv_remain > 0) and (i <= (inv_expire_month - 1)):
                str_remain = out_str[i] - begin_inv_remain     # 这个月还欠多少货
                begin_inv_remain = 0
            else:
                str_remain = out_str[i]
            for j in range(len(stw_remain)):
                if i > (inv_expire_month + j):
                    pass                             # 如果到了8月，1月进货的stw已经不能用了
                else:
                    delta = min(str_remain, stw_remain[j])   # 这个月的剩余需求和当前月的进货余量            
                    str_remain -= delta
                    stw_remain[j] -= delta
                if j <= i:
                    stw_remain_list.append(stw_remain[j])    #  大于当前月的进货不能用
                else:
                    stw_remain_list.append(None) 
                if j == i:
                    str_remain_list.append(str_remain)       # 用于检验这个月str需求是否全被满足了
        stw_remain_monthly.append(stw_remain_list)
        begin_inv_remain_monthly.append(begin_inv_remain)
    #     return stw_remain_monthly, begin_inv_remain_monthly, str_remain_list
    stw_replenish_df = make_monthly_horizontal_df(stw, 'STW(+)')
    str_remain_df = make_monthly_horizontal_df(str_remain_list, 'str_remain')
    out_str_df = make_monthly_horizontal_df(out_str, 'STR(-)')
    begin_inv_remain_df = make_monthly_horizontal_df(begin_inv_remain_monthly, 'openning_balance')
    stw_balance_df = pd.DataFrame(stw_remain_monthly, index=range(1, simu_months + 1), columns=range(1, simu_months + 1)).T
    stw_balance_df = stw_balance_df.add_prefix('month_')
    stw_balance_df.index = stw_balance_df.index.map(lambda x: str(x) + '_STW_balance')
    output = pd.concat([stw_replenish_df, out_str_df, begin_inv_remain_df, stw_balance_df, str_remain_df])
    year_begin_inv_balance = pd.DataFrame([begin_inv] + [None]*(len(output)-1), index=output.index, columns=['Openning Balance'])
    return pd.concat([year_begin_inv_balance, output], axis=1)

In [21]:
def get_result_df(begin_inv, stw, ws_str, expire_month, simu_months):
    ws_begin_inv,sku_hl_mapping_inv = get_initial_inv(begin_inv, **ws_params_dict_inv)   # 获取年期初库存
    ws_stw_lst, _ = get_num_list(stw, **ws_params_dict)                  # 获取每个月的进货量stw
    ws_str_lst,sku_hl_mapping_str = get_num_list(ws_str, **ws_params_dict_str)           # 获取每个月的出货量str
#         if min(len(ws_stw_lst), len(ws_str_lst)) < simu_months:
#             continue
    ws_begin_inv = 0 if len(ws_begin_inv) == 0 else ws_begin_inv.values[0]
    result_df = get_stw_monthly_balance(begin_inv=ws_begin_inv, stw=ws_stw_lst, out_str=ws_str_lst, inv_expire_month=expire_month, simu_months=simu_months)
    return result_df,sku_hl_mapping_inv,sku_hl_mapping_str

In [26]:
result_df=result_df_dict['W00013686']
pd.DataFrame([f"brand: {brand}_expire_month:{brand_expire_month}"] + [None]*(len(result_df.columns)-1)).T

,0,1,2,3,4,5,6,7,8,9,...,37,38,39,40,41,42,43,44,45,46
0,a,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None


In [29]:
def generate_fifo_inventory_balance(used_ws_dict, begin_inv, stw, ws_str, col='stw_qty', inv_expire_month=6, simu_months=12):
    """
        used_ws_dict: 要模拟库存的经销商
        begin_inv:年初库存
        stw: 所有经销商的stw数据
        ws_str: 所有经销商的str出货数据
        col: stw_qty箱数 or hl百升数; 算库存箱数还是百升数
        inv_expire_month: 多少个月后酒库存的就过期了,如果是int类型则所有sku默认一样的，如果是个dict，则是by brand有效期不同
        simu_months: 模拟的月份数，默认是1年即12个月
    """
    dt_string = datetime.now().date().strftime("%m%d")
    if isinstance(inv_expire_month, int):
        file = pd.ExcelWriter(f'result_df/result_data_{col}_{dt_string}_{inv_expire_month}.xlsx')
    else:
        file = pd.ExcelWriter(f'result_df/result_data_{col}_{dt_string}_by_brand_expired.xlsx')
    
    if col == 'hl':                                      # 通过洗sku的名字算hl
        ws_params_dict_inv['agg_col'] = 'hl'
        ws_params_dict['agg_col'] = 'stw_hl'
        ws_params_dict_str['agg_col'] = 'hl'
    elif col == 'hl_src':                                # 取数算的百升
        ws_params_dict_inv['agg_col'] = 'inv_hl'
        ws_params_dict['agg_col'] = 'stw_hl'
        ws_params_dict_str['agg_col'] = 'str_hl'
    else:                                                # qty箱数
        pass    
        
    sku_hl_mapping_df = pd.DataFrame()
    result_df_dict = {}
    for ws_code, payer_code in used_ws_dict.items():
        print(f"ws_code:payer_code {ws_code}:{payer_code}")
        #     ws_stw_lst = ws_stw_agg.to_list()
        ws_params_dict_inv['ws_payer_code'] = ws_code
        ws_params_dict['ws_payer_code'] = payer_code
        ws_params_dict_str['ws_payer_code'] = payer_code
        if isinstance(inv_expire_month, int):    # 所有SKU/brand默认有相同的保鲜期
            result_df = get_result_df(begin_inv, stw, ws_str, inv_expire_month, simu_months)
        else:    # 一个brand 一个结果，最后加起来汇总成一个经销商的
            result_df = pd.DataFrame()
            startrow = 0
            for brand, brand_expire_month in inv_expire_month.items():
                begin_inv_brand = begin_inv[begin_inv[ws_params_dict_inv['brand_col']] == brand]
                stw_brand = stw[stw[ws_params_dict['brand_col']] == brand]
                ws_str_brand = ws_str[ws_str[ws_params_dict_str['brand_col']] == brand]
                result_df_brand,sku_hl_mapping_inv,sku_hl_mapping_str = get_result_df(begin_inv_brand, stw_brand, ws_str_brand, brand_expire_month, simu_months)
                result_df = result_df.combine(result_df_brand, lambda s1, s2: s1.fillna(0) + s2.fillna(0))
                descrip = pd.DataFrame([f"brand: {brand}_expire_month:{brand_expire_month}"] + [None]*(len(result_df_brand.columns)-1)).T
                descrip.to_excel(file, sheet_name=f"{ws_code}_by_brand", startrow=startrow, index=False, header=False)
                result_df_brand.to_excel(file, sheet_name=f"{ws_code}_by_brand", startrow=startrow+2)
                startrow += len(result_df_brand) + 4
        result_df.to_excel(file, sheet_name=ws_code)
        result_df_dict[ws_code] = result_df
        if len(sku_hl_mapping_inv) > 0:
            sku_hl_mapping_inv['type'] = 'inv'
            sku_hl_mapping_inv['ws_code'] = ws_code
            sku_hl_mapping_df = pd.concat([sku_hl_mapping_df, sku_hl_mapping_inv])
        if len(sku_hl_mapping_str) > 0:
            sku_hl_mapping_str['type'] = 'str'
            sku_hl_mapping_str['ws_code'] = ws_code
            sku_hl_mapping_df = pd.concat([sku_hl_mapping_df, sku_hl_mapping_str])
    if len(sku_hl_mapping_df) > 0:
        sku_hl_mapping_df.to_excel(file, sheet_name='sku_hl_mapping')
    file.close()
    return result_df_dict

In [43]:
# a = result_df_dict['W00013686']
# a.insert(0, 'bu', 'BUN')
a.index

Index(['STW(+)', 'STR(-)', 'openning_balance', '1_STW_balance',
       '2_STW_balance', '3_STW_balance', '4_STW_balance', '5_STW_balance',
       '6_STW_balance', '7_STW_balance', '8_STW_balance', '9_STW_balance',
       '10_STW_balance', '11_STW_balance', '12_STW_balance', '13_STW_balance',
       '14_STW_balance', '15_STW_balance', '16_STW_balance', '17_STW_balance',
       '18_STW_balance', '19_STW_balance', '20_STW_balance', '21_STW_balance',
       '22_STW_balance', '23_STW_balance', '24_STW_balance', '25_STW_balance',
       '26_STW_balance', '27_STW_balance', '28_STW_balance', '29_STW_balance',
       '30_STW_balance', '31_STW_balance', '32_STW_balance', '33_STW_balance',
       '34_STW_balance', '35_STW_balance', '36_STW_balance', '37_STW_balance',
       '38_STW_balance', '39_STW_balance', '40_STW_balance', '41_STW_balance',
       '42_STW_balance', '43_STW_balance', '44_STW_balance', '45_STW_balance',
       '46_STW_balance', 'str_remain'],
      dtype='object')

In [ ]:
b = pd.DataFrame(index=[])

数据的字段名

In [6]:
with open('config.yml', 'r', encoding='utf-8') as file:
    params_dict = yaml.safe_load(file)
ws_params_dict_inv = params_dict['ws_params_dict_inv']  # inv数据使用列名
ws_params_dict = params_dict['ws_params_dict']     # stw数据 列名
ws_params_dict_str = params_dict['ws_params_dict_str']   # str列名

获取数据

In [7]:
used_wccs_sku_list, sku_filtered, product_id_mapping = get_used_brand_family() # sku_filtered是过滤了四个brand的sku mapping表；product_id_mapping是全量的sku hl mapping表
begin_inv = get_inv_data(used_wccs_sku_list, sku_filtered, used_qty_col='期初库存')
stw = get_stw_data(start=202101, end=202410)
ws_str = get_str_data(product_id_mapping,start=202101, end=202410)

没对应上SKUCode/HL的wccs product id ['P02816' 'P02747' 'P02746' 'P02931' 'P03012' 'P03002' 'P02896' 'P02820'
 'P02650' 'P02843' 'P02708' 'P03001' 'P03071' 'P03020' 'P00012' 'P00007'
 'P01040' 'P00024' 'P00043' 'P00033' 'P00038' 'P00040' 'P00032' 'P00028'
 'P00020' 'P00025' 'P00015' 'P00034' 'P00036' 'P00031' 'P00029' 'P00019'
 'P00014' 'P00027' 'P00035' 'P00830' 'P00211' 'P00022' 'P00030' 'P00627'
 'P02935' 'P03006' 'P02757' 'P00315' 'P00047' 'P00045' 'P00076' 'P00046'
 'P00055' 'P00098' 'P00053' 'P00051' 'P00050' 'P00064' 'P00062' 'P00058'
 'P00059' 'P01018' 'P00061' 'P00063' 'P00867' 'P00078' 'P02791' 'P02799'
 'P00073' 'P00074' 'P00056' 'P00065' 'P00067' 'P00072' 'P00057' 'P00233'
 'P00408' 'P00329' 'P00307' 'P00267' 'P00314' 'P00272' 'P00317' 'P00359'
 'P00444' 'P00481' 'P00343' 'P00704' 'P00249' 'P00301' 'P00254' 'P00260'
 'P00327' 'P00273' 'P00278' 'P00446' 'P00299' 'P00316' 'P00225' 'P00633'
 'P00401' 'P00631' 'P00469' 'P00295' 'P00653' 'P00558' 'P00231' 'P00854'
 'P00585' 'P00308' '

In [12]:
ws_str['Brand_Family_ComDB'].unique()

array(['BUD', 'HBO', 'HBI', 'C&S', 'HEC', 'S-P Others', 'HKOW'],
      dtype=object)

In [13]:
stw['brand_family_comdb'].unique()

array(['C&S', 'BUD', 'HEC', 'HBI', 'S-P Others', 'HBO', 'HKOW'],
      dtype=object)

In [8]:
month_cnt = len(stw['billingyearmonth'].unique())      # 2021.1到24年10月是46个月；每个经销商都要46个月
print(f"stw月份数：{month_cnt}")
print(f"str月份数：{len(ws_str['month'].unique())}")
stw_yr_month_index = stw['billingyearmonth'].drop_duplicates().sort_values()
# str_yr_month_index = ws_str['month'].drop_duplicates().sort_values()   # 长得是一样的

stw月份数：46
str月份数：46


In [30]:
used_ws_dict = {'W00013686': 30018687, 'W00014676':30019378, 'W00008121': 30012653}
inv_unit = 'hl_src'         # stw_qty 或其他值，都是指默认值，箱数/////hl_src是原始数据带的百升数/////hl是根据sku洗出来的百升数
# generate_fifo_inventory_balance(used_ws_dict, begin_inv, stw, ws_str, col='stw_qty',inv_expire_month=inv_expire_month, simu_months=month_cnt)
inv_expire_month = 9    # 库存9个月后失效，之前是6个月
brand_expire_month_dict = {'BUD': 9, 'HBI': 9, 'HBO': 9, 'HKOW':9}
# 默认月份一样
# result_df_dict = generate_fifo_inventory_balance(used_ws_dict, begin_inv, stw, ws_str, col='hl_src',inv_expire_month=inv_expire_month, simu_months=month_cnt)
# 每个brand的保质期都不一样
result_df_dict = generate_fifo_inventory_balance(used_ws_dict, begin_inv, stw, ws_str, col='hl_src',inv_expire_month=brand_expire_month_dict, simu_months=month_cnt)

ws_code:payer_code W00013686:30018687
ws_code:payer_code W00014676:30019378
ws_code:payer_code W00008121:30012653


In [11]:
a = pd.DataFrame(dict(a=[1,2,3,4,None], b=[None,2,3,4,1], c=[1,None,2,3,1]))
b = pd.DataFrame(dict(a=[1,2,3,4,5], b=[1,2,3,4,1], c=[1,2,2,3,1]))

In [15]:
pd.isnull(None)

True

In [17]:
a.combine(b, lambda s1, s2: s1.fillna(0) + s2.fillna(0))

,a,b,c
0,2.0,1.0,2.0
1,4.0,4.0,2.0
2,6.0,6.0,4.0
3,8.0,8.0,6.0
4,5.0,2.0,2.0


### analysis
1. write_off_amt_prct: write off的量/进货的量
2. write_off_month_cnt: 有write off量的月份数
3. write_off_month_prct: 2 / 进货月份数（包括openning）

In [33]:
write_off_rank = []
for ws_code in used_ws_dict.keys():
    result_df = result_df_dict[ws_code]
    in_stw_df = result_df.loc['STW(+)', :][:-(inv_expire_month + 1)]   # 最后1+inv_expire_month不看，因为没过期呢，也不需要参与分母计算
    total_in_stw = in_stw_df.sum()         # 分母，进货的量
    balance_df_at_last = result_df.iloc[2:-1, -1]
    expired_balance_df = balance_df_at_last[:-(inv_expire_month + 1)]
    write_off_stw = expired_balance_df.sum()    #分子
    write_off_amt_prct = write_off_stw / total_in_stw if total_in_stw > 0 else None          # write off的量 / 进货的量
    write_off_month_cnt = expired_balance_df[expired_balance_df > 0].shape[0]
    write_off_month_prct = write_off_month_cnt / len(in_stw_df[in_stw_df > 0]) if len(in_stw_df[in_stw_df > 0]) > 0 else None
    write_off_rank.append(dict(ws_code=ws_code, total_in_stw=total_in_stw, write_off_stw=write_off_stw, write_off_amt_prct=write_off_amt_prct, 
                               write_off_month_cnt=write_off_month_cnt, write_off_month_prct=write_off_month_prct))
ws_rank_df = pd.DataFrame(write_off_rank)

In [44]:
result_df

,Openning Balance,month_1,month_2,month_3,month_4,month_5,month_6,month_7,month_8,month_9,...,month_37,month_38,month_39,month_40,month_41,month_42,month_43,month_44,month_45,month_46
STW(+),18656.16864,14472.10900,16435.99000,26586.06800,28221.35600,26734.27400,36576.66100,31631.58900,38074.46000,22597.85600,...,6135.63900,7965.23600,28337.58400,12649.59400,31134.85200,28629.59500,24084.16500,25566.75800,23805.34700,5428.43500
STR(-),0.00000,15655.68600,14753.99880,28581.66984,12093.89208,12912.45648,52195.10040,29035.54920,31417.40280,31316.96760,...,19046.12304,16573.52088,21103.11864,16723.30848,34494.39384,25283.98512,27861.61296,15213.71568,21300.08688,21858.48264
openning_balance,0.00000,4212.84264,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,...,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000
1_STW_balance,0.00000,13259.74900,2718.59284,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,...,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000
2_STW_balance,0.00000,0.00000,16435.99000,2794.74000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,...,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000
3_STW_balance,0.00000,0.00000,0.00000,14364.24100,6024.48892,0.00000,0.00000,0.00000,0.00000,0.00000,...,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000
4_STW_balance,0.00000,0.00000,0.00000,0.00000,27261.95600,21965.36844,0.00000,0.00000,0.00000,0.00000,...,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000
5_STW_balance,0.00000,0.00000,0.00000,0.00000,0.00000,25142.89400,0.00000,0.00000,0.00000,0.00000,...,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000
6_STW_balance,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,31489.82304,2454.27384,0.00000,0.00000,...,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000
7_STW_balance,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,31631.58900,2668.46004,0.00000,...,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000


In [34]:
ws_rank_df
+ BU + region + month(24年10月) + ws_name
+ 最后期末的有效valid_stw_balance

,ws_code,total_in_stw,write_off_stw,write_off_amt_prct,write_off_month_cnt,write_off_month_prct
0,W00013686,35814.28880,333.66,0.009316,3,0.081081
1,W00014676,0.00000,0.00,NaN,0,NaN
2,W00008121,819416.94064,0.00,0.000000,0,0.000000


In [42]:
ws_rank_df.columns

Index(['ws_code', 'total_in_stw', 'write_off_stw', 'write_off_amt_prct',
       'write_off_month_cnt', 'write_off_month_prct'],
      dtype='object')

In [21]:
# 查数

for ws_code in used_ws_dict.keys():
    if ws_code in begin_inv['经销商代码'].unique():
        pass
    else:
        print(ws_code)
        
for ws_code in used_ws_dict.values():
    if ws_code in stw['PayerCode'].unique():
        pass
    else:
        print(ws_code)
        
for ws_code in used_ws_dict.values():
    if ws_code in ws_str['payercode'].unique():
        pass
    else:
        print(ws_code)

In [121]:
# stw_balance_df = pd.DataFrame(stw_remain_monthly, index=range(1,13), columns=range(1,13)).T
# stw_balance_df = stw_balance_df.add_prefix('month_')
# stw_balance_df.index = stw_balance_df.index.map(lambda x: 'stw_balance_' + str(x))
# stw_balance_df

测试逻辑

In [140]:
"""
begin_inv:202212期末库存=2023年1月期初库存
stw: 2023年12个月的stw进货量
out_str:2023年12个月的出货量

result: 期初库存每个月的结余
"""


# 测试数据
begin_inv = 80
stw = [80]*2 + [100]*10     # 进货
out_str = [80]*7 + [80]*5

begin_inv_remain = begin_inv
stw_remain = stw.copy()             #  截止到当前月每个月的进货量还剩多少                
stw_remain_monthly = []             # 每个月的结果合集
begin_inv_remain_monthly = []       # 每个月的结果合集
#     begin_inv_remain_lst = [None] * 12
#     stw_remain_list = []        # 每个月进货stw的剩余
str_remain_list = []

for i in range(12):
    stw_remain_list = []
    if (out_str[i] <= begin_inv_remain) and (i <= 5):     # 期初库存能扣到6月
        begin_inv_remain -= out_str[i]  # 期初（年初）库存的剩余
        stw_remain_list = stw_remain.copy()     # 每个月stw的剩余
        stw_remain_list = [stw_remain_list[stw_month] if stw_month <= i else None for stw_month in range(12) ]
        str_remain = 0
        str_remain_list.append(str_remain)
    else:
        if (begin_inv_remain > 0) and (i <= 5):
            str_remain = out_str[i] - begin_inv_remain     # 这个月还欠多少货
            begin_inv_remain = 0
        else:
            str_remain = out_str[i]
        for j in range(len(stw_remain)):
            if i > (6+j):
                pass                             # 如果到了8月，1月进货的stw已经不能用了
            else:
                delta = min(str_remain, stw_remain[j])   # 这个月的剩余需求和当前月的进货余量            
                str_remain -= delta
                stw_remain[j] -= delta
            if j <= i:
                stw_remain_list.append(stw_remain[j])
            else:
                stw_remain_list.append(None)
            if j == i:
                str_remain_list.append(str_remain)
    stw_remain_monthly.append(stw_remain_list)
    begin_inv_remain_monthly.append(begin_inv_remain)
#     return stw_remain_monthly, begin_inv_remain_monthly, str_remain_list
stw_replenish_df = make_monthly_horizontal_df(stw, 'stw_in')
str_remain_df = make_monthly_horizontal_df(str_remain_list, 'str_remain')
out_str_df = make_monthly_horizontal_df(out_str, 'out_str')
begin_inv_remain_df = make_monthly_horizontal_df(begin_inv_remain_monthly, 'openning_balance')
stw_balance_df = pd.DataFrame(stw_remain_monthly, index=range(1,13), columns=range(1, 13)).T
stw_balance_df = stw_balance_df.add_prefix('month_')
stw_balance_df.index = stw_balance_df.index.map(lambda x: str(x) + '_STW_balance')
pd.concat([stw_replenish_df, out_str_df, begin_inv_remain_df, stw_balance_df, str_remain_df])

,month_1,month_2,month_3,month_4,month_5,month_6,month_7,month_8,month_9,month_10,month_11,month_12
stw_in,80.0,80.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0
out_str,80.0,80.0,80.0,80.0,80.0,80.0,80.0,80.0,80.0,80.0,80.0,80.0
openning_balance,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1_STW_balance,80.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2_STW_balance,NaN,80.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3_STW_balance,NaN,NaN,100.0,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4_STW_balance,NaN,NaN,NaN,100.0,40.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5_STW_balance,NaN,NaN,NaN,NaN,100.0,60.0,0.0,0.0,0.0,0.0,0.0,0.0
6_STW_balance,NaN,NaN,NaN,NaN,NaN,100.0,80.0,0.0,0.0,0.0,0.0,0.0
7_STW_balance,NaN,NaN,NaN,NaN,NaN,NaN,100.0,100.0,20.0,0.0,0.0,0.0


In [305]:
def weird_sku_name(x):
    x = re.sub(r'\s|[\d]%','',x)
    a = re.search(r'.+[度|\u4E00-\u9Fa5 ]([0-9.]+ML|[0-9.]+L)([0-9]+[X|*][0-9]+|桶|回桶).+', x)
    if a:
        ml_str = a.group(1)            
        bottle_cnt_str = a.group(2)
        liter_int = float(re.match(r'[\d.]+', ml_str).group())/1000 if 'M' in ml_str else float(re.match(r'[\d.]+', ml_str).group())
        bottle_cnt_str = bottle_cnt_str.replace("*", 'X')
        if re.search(r'([0-9]+X[0-9]+)', bottle_cnt_str):
            bottle_cnt_int = int(bottle_cnt_str.split('X')[0])*int(bottle_cnt_str.split('X')[1])
        else:
            bottle_cnt_int = 1 if '桶' in bottle_cnt_str else re.search(r'[\d]+', bottle_cnt_str).group()
#         print(x, liter_int, bottle_cnt_int)
        return ml_str, bottle_cnt_str, liter_int, bottle_cnt_int
    elif re.search(r'.+[度|\u4E00-\u9Fa5 ]([0-9.]+ML|[0-9.]+L)([[X|*][0-9]+|桶|回桶).+', x): #  范佳乐小麦13.8度1LX6手提纸箱听装-荣耀特酿
        a = re.search(r'.+[度|\u4E00-\u9Fa5 ]([0-9.]+ML|[0-9.]+L)([[X|*][0-9]+|桶|回桶).+', x)
        ml_str = a.group(1)
        bottle_cnt_str = a.group(2)
        liter_int = float(re.match(r'[\d.]+', ml_str).group())/1000 if 'M' in ml_str else float(re.match(r'[\d.]+', ml_str).group())
        bottle_cnt_str = bottle_cnt_str.replace("*", 'X')
        bottle_cnt_int = int(bottle_cnt_str.replace("X", ""))
        return ml_str, bottle_cnt_str, liter_int, bottle_cnt_int
    else:
        raise ValueError(f"wrong sku name rules {x}")

('1L', 'X6', 1.0, 6)

In [268]:

for x in weird_name_list:
    x = re.sub(r'\s|[\d]%', '', x)
    a = re.search(r'.+[度|\u4E00-\u9Fa5 ]([0-9.]+ML|[0-9.]+L)([0-9]+[X|*][0-9]+|桶|回桶).+', x)
    try:
        print(a.group(1))
        print(a.group(2))
    except:

        if re.search(r'.+[度|\u4E00-\u9Fa5 ]([0-9.]+ML|[0-9.]+L)([[X|*][0-9]+|桶|回桶).+', x):   #  范佳乐小麦13.8度1LX6手提纸箱听装-荣耀特酿
            b = re.search(r'.+[度|\u4E00-\u9Fa5 ]([0-9.]+ML|[0-9.]+L)([[X|*][0-9]+|桶|回桶).+', x)
            print(b.group(1))
            print(b.group(2))
        elif re.search(r'回桶', x):
            b = re.search(r'.+[度|\u4E00-\u9Fa5 ]([0-9.]+ML|[0-9.]+L)', x)  # 'CT_鹅岛印度淡色艾尔15.46度19.5L1X1回桶装'
            print(b.group(1))

        else:
            print(x)

30L
1*1
30L
桶
30L
桶
330ML
1X24
330ML
4*6
19.5L
1X1
19.5L
1X1
19.5L
1X1
20L
回桶
20L
回桶
20L
回桶
1L
X6


In [248]:
weird_name_list = ['CT_百威9.7度30L1*1桶装',
'福佳11.5度30L桶白啤',
'CT_乐飞金15.6度30L桶啤',
'时代11.35 度330ML5% 1X24纸箱OW瓶装',
'科罗娜11.3度330ML4*6纸箱OW',
'CT_鹅岛印度淡色艾尔15.46度19.5L1X1回桶装',
'CT_鹅岛醺然艾尔11.5度19.5L1X1回桶装',
'CT_鹅岛312 10.27度19.5L1X1回桶装',
'CT_拳击猫搏击者超淡色艾尔啤酒12度20L回桶装-自产',
'CT_拳击猫右勾拳德式淡色拉格啤酒10.8度20L回桶装-自产',
'CT_拳击猫美式琥珀拉格12.5度20L回桶装-自产'
'CT_拳击猫胜利之拳印度淡色艾尔15.0度20L回桶装-自产',
'范佳乐小麦13.8度1LX6手提纸箱听装-荣耀特酿']


In [ ]:
# 没有过期stw一说的
# def get_stw_monthly_balance(begin_inv, stw: list, out_str: list):
#     """
#     begin_inv:202212期末库存=2023年1月期初库存
#     stw: 2023年12个月的stw进货量
#     out_str:2023年12个月的出货量
    
#     result: 期初库存每个月的结余
#     """
#     begin_inv_remain = begin_inv
#     #     测试数据
#     #     stw = [80]*2 + [60]*10     # 进货
#     #     out_str = [100]*3 + [80]*9

#     stw_remain = stw.copy()             #  截止到当前月每个月的进货量还剩多少                
#     stw_remain_monthly = []             # 每个月的结果合集
#     begin_inv_remain_monthly = []       # 每个月的结果合集
# #     begin_inv_remain_lst = [None] * 12
# #     stw_remain_list = []        # 每个月进货stw的剩余
#     str_remain_list = []

#     for i in range(12):
#         stw_remain_list = []
#         if out_str[i] <= begin_inv_remain:
#             begin_inv_remain -= out_str[i]  # 期初（年初）库存的剩余
#             stw_remain_list = stw_remain.copy()     # 每个月stw的剩余
#             stw_remain_list = [stw_remain_list[stw_month] if stw_month <= i else None for stw_month in range(12) ]
#             str_remain = 0
#             str_remain_list.append(str_remain)
#         else:
#             if begin_inv_remain > 0:
#                 str_remain = out_str[i] - begin_inv_remain     # 这个月还欠多少货
#                 begin_inv_remain = 0
#             else:
#                 str_remain = out_str[i]
#             for j in range(len(stw_remain)):
#                 delta = min(str_remain, stw_remain[j])   # 这个月的剩余需求和当前月的进货余量            
#                 str_remain -= delta
#                 stw_remain[j] -= delta
#                 if j <= i:
#                     stw_remain_list.append(stw_remain[j])
#                 else:
#                     stw_remain_list.append(None)
#                 if j == i:
#                     str_remain_list.append(str_remain)
#         stw_remain_monthly.append(stw_remain_list)
#         begin_inv_remain_monthly.append(begin_inv_remain)
# #     return stw_remain_monthly, begin_inv_remain_monthly, str_remain_list
#     stw_replenish_df = make_monthly_horizontal_df(stw, 'stw_in')
#     str_remain_df = make_monthly_horizontal_df(str_remain_list, 'str_remain')
#     out_str_df = make_monthly_horizontal_df(out_str, 'out_str')
#     begin_inv_remain_df = make_monthly_horizontal_df(begin_inv_remain_monthly, 'openning_balance')
#     stw_balance_df = pd.DataFrame(stw_remain_monthly, index=range(1,13), columns=range(1, 13)).T
#     stw_balance_df = stw_balance_df.add_prefix('month_')
#     stw_balance_df.index = stw_balance_df.index.map(lambda x: str(x) + '_STW_balance')    
#     return pd.concat([stw_replenish_df, out_str_df, begin_inv_remain_df, stw_balance_df, str_remain_df])

In [70]:
a = {'a':1,'b':3}
a.get('a','aa')
a

{'a': 1, 'b': 3}